# <center> Desafio 01 - VAI Academy </center> 
## <center> Estudo de Caso </center>
### <center> Otimização de Captura de Leads </center>

## Importando Bibliotecas

In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math

## Lendo arquivos Excel

In [23]:
deals = pd.read_excel('vcommerce_deals.xlsx', engine='openpyxl')
leads = pd.read_excel('vcommerce_marketing_qualified_leads.xlsx', engine='openpyxl')
sellers = pd.read_excel('vcommerce_sellers.xlsx', engine='openpyxl')

In [29]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   mql_id                         842 non-null    object        
 1   seller_id                      842 non-null    object        
 2   won_date                       434 non-null    datetime64[ns]
 3   business_segment               842 non-null    object        
 4   lead_type                      842 non-null    object        
 5   lead_behaviour_profile         842 non-null    object        
 6   has_company                    63 non-null     object        
 7   has_gtin                       64 non-null     object        
 8   average_stock                  66 non-null     object        
 9   business_type                  832 non-null    object        
 10  declared_product_catalog_size  842 non-null    float64       
 11  declared_monthly_re

In [31]:
leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   mql_id              8000 non-null   object        
 1   first_contact_date  8000 non-null   datetime64[ns]
 2   origin              7940 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 187.6+ KB


In [30]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


### Taxa de conversão de leads por segmento

### Mesclando tabelas deals e leads

In [42]:
leads_m_deals = pd.merge(deals, leads, how='left')
df = pd.merge(leads_m_deals, sellers, how='left')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 0 to 841
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   mql_id                         842 non-null    object        
 1   seller_id                      842 non-null    object        
 2   won_date                       434 non-null    datetime64[ns]
 3   business_segment               842 non-null    object        
 4   lead_type                      842 non-null    object        
 5   lead_behaviour_profile         842 non-null    object        
 6   has_company                    63 non-null     object        
 7   has_gtin                       64 non-null     object        
 8   average_stock                  66 non-null     object        
 9   business_type                  832 non-null    object        
 10  declared_product_catalog_size  842 non-null    float64       
 11  declared_monthly_re

In [47]:
closed_leads = df.query('closed_deal == "Yes"')
closed_leads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434 entries, 0 to 841
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   mql_id                         434 non-null    object        
 1   seller_id                      434 non-null    object        
 2   won_date                       434 non-null    datetime64[ns]
 3   business_segment               434 non-null    object        
 4   lead_type                      434 non-null    object        
 5   lead_behaviour_profile         434 non-null    object        
 6   has_company                    35 non-null     object        
 7   has_gtin                       34 non-null     object        
 8   average_stock                  38 non-null     object        
 9   business_type                  428 non-null    object        
 10  declared_product_catalog_size  434 non-null    float64       
 11  declared_monthly_re

In [59]:
converted_customers_per_segment = pd.crosstab(index=closed_leads['business_segment'], columns='converted_count').reset_index()
converted_customers_per_segment

col_0,business_segment,converted_count
0,audio_video_electronics,35
1,baby_kids,12
2,bags_backpacks,11
3,bed_bath_table,11
4,car_accessories,36
5,computers_phones_mobiles,27
6,construction_tools_house_garden,36
7,food_drink,12
8,food_suplement,12
9,health_beauty_fashion,61


In [61]:
customers_per_segment = pd.crosstab(index=df['business_segment'], columns='total_count').reset_index()
customers_per_segment

col_0,business_segment,total_count
0,audio_video_electronics,64
1,baby_kids,30
2,bags_backpacks,22
3,bed_bath_table,22
4,car_accessories,77
5,computers_phones_mobiles,47
6,construction_tools_house_garden,69
7,food_drink,26
8,food_suplement,28
9,health_beauty_fashion,130


In [66]:
m = pd.merge(customers_per_segment, converted_customers_per_segment, how='inner')
m

col_0,business_segment,total_count,converted_count
0,audio_video_electronics,64,35
1,baby_kids,30,12
2,bags_backpacks,22,11
3,bed_bath_table,22,11
4,car_accessories,77,36
5,computers_phones_mobiles,47,27
6,construction_tools_house_garden,69,36
7,food_drink,26,12
8,food_suplement,28,12
9,health_beauty_fashion,130,61


In [71]:
m['conv_rate'] = m['converted_count']/m['total_count']*100
m

col_0,business_segment,total_count,converted_count,conv_rate
0,audio_video_electronics,64,35,54.687500
1,baby_kids,30,12,40.000000
2,bags_backpacks,22,11,50.000000
3,bed_bath_table,22,11,50.000000
4,car_accessories,77,36,46.753247
5,computers_phones_mobiles,47,27,57.446809
6,construction_tools_house_garden,69,36,52.173913
7,food_drink,26,12,46.153846
8,food_suplement,28,12,42.857143
9,health_beauty_fashion,130,61,46.923077


In [81]:
max_segment = m['conv_rate'].max()
min_segment = m['conv_rate'].min()

"Floating point {0:.2f}".format(345.7916732)
print('Segmento com maior taxa de conversão: {0:.2f}'.format(max_segment))
print('Segmento com menor taxa de conversão: {0:.2f}'.format(min_segment))

Segmento com maior taxa de conversão: 58.33
Segmento com menor taxa de conversão: 40.00


In [85]:
closed_leads = df.query('closed_deal=="Yes"')
closed_leads.head()

,mql_id,seller_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,closed_deal,first_contact_date,origin,seller_zip_code_prefix,seller_city,seller_state
0,000dd3543ac84d906eae52e7c779bb2a,4e0d109ccfba9a1151c103049880c857,2018-04-24 03:00:00,home_decor_furniture,industry,cat,NaN,NaN,NaN,manufacturer,0.0,0.0,Yes,2018-04-05,organic_search,25550,sao joao de meriti,RJ
2,00dc5000951c268cc8655294daf67b1b,bbf9ad41dca6603e614efcdad7aab8c4,2018-07-10 11:34:50,home_decor_furniture,online,cat,False,False,20-50,reseller,0.0,0.0,Yes,2018-03-26,paid_search,60348,fortaleza,CE
5,01409f7f7f2bd5ec0c1755b362e9db6e,8d92f3ea807b89465643c219455e7369,2018-03-06 18:14:08,health_beauty_fashion,online,cat,NaN,NaN,NaN,reseller,0.0,0.0,Yes,2018-01-30,other_publicities,3052,sao paulo,SP
10,0338bf13624f52beaca91ec4a23c860c,38e6dada03429a47197d5d584d793b41,2018-04-30 13:30:05,home_decor_furniture,online,eagle,NaN,NaN,NaN,NaN,0.0,0.0,Yes,2018-04-05,display,4763,sao paulo,SP
12,03ba283fed2772f4e477eea4f0f236b8,7901646fdd36a55f564ffaf2dbccaaf7,2018-03-16 13:57:28,food_drink,online,cat,NaN,NaN,NaN,manufacturer,0.0,0.0,Yes,2018-02-26,organic_search,20021,rio de janeiro,RJ


In [88]:
closed_leads['cont_time'] = datetime(closed_leads.won_date) - datetime(closed_leads.first_contact_date)

TypeError: 'Series' object cannot be interpreted as an integer